# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.75it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.62it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.34it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]



...

 65%|██████▌   | 15/23 [00:03<00:01,  5.99it/s]

 74%|███████▍  | 17/23 [00:03<00:00,  6.07it/s]

 83%|████████▎ | 19/23 [00:03<00:00,  6.15it/s]

 91%|█████████▏| 21/23 [00:04<00:00,  6.27it/s]

100%|██████████| 23/23 [00:04<00:00,  5.10it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sophia and I am a Senior, majoring in Human Communication. I am excited to be a part of the Student Ambassador program this year and look forward to connecting with you!
As a communication major, I have a passion for connecting people and facilitating meaningful interactions. I believe that effective communication is the foundation of any successful relationship, and I am eager to help make that happen for our students here at WSU.
Outside of academics, I love to stay active and can often be found practicing yoga or going for a hike. I also enjoy trying out new recipes in the kitchen and exploring the local music scene.
As a Student Ambassador, I am
Prompt: The president of the United States is
Generated text:  both the head of state and the head of government. The president serves a four-year term and is elected by the people through the Electoral College. The president has the power to veto laws passed by Congress, although Congress can over

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Duncan

 Duncan Hart

 Duncan Hartley

 Duncan Hartley and

...

 in the hands of the humans who program it
The future of AI is in the hands of the humans who program it
The future of artificial intelligence (AI) is shaped by the humans who design, develop, and deploy AI systems. These developers and programmers have the power to influence the development of AI and its impact on society.
AI development is a rapidly evolving field, with new techniques, tools, and applications emerging every year. However, the rapid progress in AI has also raised concerns about the potential risks and challenges associated with its development and deployment.
The future of AI is shaped by the decisions and choices made by

 in the hands of the humans who program it
The future of AI is in the hands of the humans who program it
The future of artificial intelligence (AI) is shaped by the humans who design, develop, and deploy AI systems. These developers and programmers have the power to influence the development of AI and its impact on society.
AI development is a rapidly evolving field, with new techniques, tools, and applications emerging every year. However, the rapid progress in AI has also raised concerns about the potential risks and challenges associated with its development and deployment.
The future of AI is shaped by the decisions and choices made by developers

 in the hands of the humans who program it
The future of AI is in the hands of the humans who program it
The future of artificial intelligence (AI) is shaped by the humans who design, develop, and deploy AI systems. These developers and programmers have the power to influence the development of AI and its impact on society.
AI development is a rapidly evolving field, with new techniques, tools, and applications emerging every year. However, the rapid progress in AI has also raised concerns about the potential risks and challenges associated with its development and deployment.
The future of AI is shaped by the decisions and choices made by developers,

 in the hands of the humans who program it
The future of AI is in the hands of the humans who program it
The future of artificial intelligence (AI) is shaped by the humans who design, develop, and deploy AI systems. These developers and programmers have the power to influence the development of AI and its impact on society.
AI development is a rapidly evolving field, with new techniques, tools, and applications emerging every year. However, the rapid progress in AI has also raised concerns about the potential risks and challenges associated with its development and deployment.
The future of AI is shaped by the decisions and choices made by developers, who

 in the hands of the humans who program it
The future of AI is in the hands of the humans who program it
The future of artificial intelligence (AI) is shaped by the humans who design, develop, and deploy AI systems. These developers and programmers have the power to influence the development of AI and its impact on society.
AI development is a rapidly evolving field, with new techniques, tools, and applications emerging every year. However, the rapid progress in AI has also raised concerns about the potential risks and challenges associated with its development and deployment.
The future of AI is shaped by the decisions and choices made by developers, who must

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Jo and I am a volunteer with the Waltham Forest Volunteer Centre. I am 51 and have been a volunteer for over 5 years. I was previously a civil servant and before that worked in retail management. I decided to volunteer after my children left home and I was looking for a way to give something back to the community.
I have volunteered for several different organisations, including Age UK, a community garden and a charity that supports vulnerable adults. I currently volunteer at a local library, helping with book sales and providing information to visitors. I also help out with a community project that supports refugees and asylum seekers.
Volunteering has

Prompt: The capital of France is
Generated text:  known for its beautiful gardens and parks. Paris has a number of famous parks and gardens that are worth visiting, including the Luxembourg Gardens and the Tuileries Garden. These parks are not only beautiful but also offer a peaceful escape f

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Pa

 Paola

 Paola G

 Paola Gó

...

 exciting, but also raises concerns about the potential impact on employment and society. Understanding the implications of AI and its development is crucial to addressing these concerns.
AI has been a focus of research and development for several decades, but its applications and capabilities have accelerated rapidly in recent years. AI has become a fundamental part of the digital infrastructure, transforming industries, products, and services.
AI can be categorized into different types, including:
1. Narrow or weak AI: AI that is designed to perform a specific task, such as image recognition, language translation, or playing chess.
2. General or strong AI: AI that can perform

 exciting, but also raises concerns about the potential impact on employment and society. Understanding the implications of AI and its development is crucial to addressing these concerns.
AI has been a focus of research and development for several decades, but its applications and capabilities have accelerated rapidly in recent years. AI has become a fundamental part of the digital infrastructure, transforming industries, products, and services.
AI can be categorized into different types, including:
1. Narrow or weak AI: AI that is designed to perform a specific task, such as image recognition, language translation, or playing chess.
2. General or strong AI: AI that can perform any

 exciting, but also raises concerns about the potential impact on employment and society. Understanding the implications of AI and its development is crucial to addressing these concerns.
AI has been a focus of research and development for several decades, but its applications and capabilities have accelerated rapidly in recent years. AI has become a fundamental part of the digital infrastructure, transforming industries, products, and services.
AI can be categorized into different types, including:
1. Narrow or weak AI: AI that is designed to perform a specific task, such as image recognition, language translation, or playing chess.
2. General or strong AI: AI that can perform any intellectual

 exciting, but also raises concerns about the potential impact on employment and society. Understanding the implications of AI and its development is crucial to addressing these concerns.
AI has been a focus of research and development for several decades, but its applications and capabilities have accelerated rapidly in recent years. AI has become a fundamental part of the digital infrastructure, transforming industries, products, and services.
AI can be categorized into different types, including:
1. Narrow or weak AI: AI that is designed to perform a specific task, such as image recognition, language translation, or playing chess.
2. General or strong AI: AI that can perform any intellectual task

 exciting, but also raises concerns about the potential impact on employment and society. Understanding the implications of AI and its development is crucial to addressing these concerns.
AI has been a focus of research and development for several decades, but its applications and capabilities have accelerated rapidly in recent years. AI has become a fundamental part of the digital infrastructure, transforming industries, products, and services.
AI can be categorized into different types, including:
1. Narrow or weak AI: AI that is designed to perform a specific task, such as image recognition, language translation, or playing chess.
2. General or strong AI: AI that can perform any intellectual task that

In [6]:
llm.shutdown()